# Clean lines

In [14]:
#Import libaries 
import pandas as pd
import numpy as np 
import os
import gensim
import re
import contractions

# set directoru 
os.getcwd()
os.chdir('/Users/Sofie/Disney Thesis')

# import clean scripts
all_scripts = pd.read_pickle("all_scripts_cleaned_total_27apr.pickle")

In [15]:
# character names
character_names = list(all_scripts['Character'].unique())

other_names = ["dumbo", "lightyear", "fredricksen", 
                "omalley", "madrigal", "jack", "bruno", "hen", "wen", "tod","cauldron"]
character_names.extend(other_names)

In [16]:
# replace repeated characters occuring three times or more with one character
clean_lines_0 = []
for line in all_scripts['Line']:
    
    line = line.lower() # lower lines 
    line = re.sub(r'([a-zA-Z])\1{2,}', r'\1', line)
    
    # Replace phrases 
    line = re.sub("ooh", "ohh", line)
    line = re.sub("aah", "ahh", line)
    line = re.sub("ahhh", "ahh", line)
    line = re.sub("comin", "coming", line)
    line = re.sub("c'mon", "come on", line)
    
    clean_lines_0.append(line)
    
all_scripts['Clean_Line_all'] = clean_lines_0

In [17]:
# clean html 
clean_lines = []
for line in all_scripts['Clean_Line_all']:
    line = str(line)
    if "=" in line:
        line = line.split("<")
        line = line[0]
        line = re.sub("\(", "",line)
    elif "<" in line:
        line = line.split("<!")
        line = line[0]
        line = re.sub("moana/transcript", "",line)
        line = re.sub("$1.", "",line)
        line = re.sub("<>!", "",line)
        line = re.sub("<", "",line)
    elif "<" in line:
        line = re.sub('>a', '',line) 
    elif "muses sing" in line:
        line = line.split("muses sing")
        line = line[0]
    elif "{" in line:
        line = re.sub(r' {[^}]*}','', line)
        line = line.split("{")
        line = line[0]
        
    else:
        line = line
    clean_lines.append(line)
        
all_scripts['Clean_Line_all'] = clean_lines

In [18]:
# Fixes contractions such as `you're` to you `are`
lines = []
for line in all_scripts['Clean_Line_all']:
    line = contractions.fix(line)
    lines.append(line)
all_scripts['Clean_Line_all'] = lines    

In [19]:
for line in all_scripts['Clean_Line_all']:
    if "'" in line:
        print(line)

once upon a time, there lived a lovely little princess named snow white. her vain and wicked stepmother, the queen, feared that some day snow white's beauty would surpass her own. then she dressed the little princess in rags and forced her to work as a scullery maid.
each day, the vain queen consulted her magic mirror, "magic mirror on the wall, who is the fairest one of all?" and as long as the mirror responded, "you are the fairest one of all". snow white was safe from the queen's cruel jealousy.
i really feel quite happy now. i am sure i will get along somehow. everything's going to be alright.
oh, it is adorable! just like a doll's house.
our house! the lit's light, er, the light's lit!
the chimney's smoking.
huh! chair's been dusted.
our window's been washed.
sink's empty. hey, someone stole our dishes!
my cup's been washed. sugar's gone.
you see? it is witch's brew.
here it comes. now's our chance. get it now!
supper's not quite ready. you will just have time to wash.
are not new

your cows cannot do muchwith slim's reward money.
there ain'ta view beneath the bluethat could ever compare
oh, yeah. lightning's ready.
bob, my oil pressure's through the roof right now. if this gets anymore exciting, they are going to have to tow me outta the booth!
he is been dinoco's golden boy for years! can he win them one last piston cup?
chick thought this was his year, bob. his chance to finally emerge from the king's shadow. but the last thing he expected was... lightning mcqueen!
mcqueen's not going into the pits!
looks like it is all gas-and-go's for mcqueen today.
normally i said a short-term gain, long-term loss, but it is sure is workin' for him. he obviously knows something we do not know.
mcqueen's blown a tire, mcqueen's blown a tire! go, go, go, go, go, go!
hey, lightning! yo, mcqueen, seriously, that was some pretty darn nice racin' out there. by me! welcome to the chick era, baby! the piston cup, it is mine, dude. it is mine. hey, fellas, how do you think i am look

In [20]:
# create function for cleaning lines

# import stopwords
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))

#not_stopwords = {'she', 'he'}  # we dont want to remove he and she
#final_stop_words = set([word for word in stop_words if word not in not_stopwords])


# import lemmatizer 
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def remove_punc(string):
    # removing all everything except a-z english letters
    temp_sent = []
    
    regex = re.compile('[^a-zA-Z]')
    temp_string = regex.sub(' ', string)
    # removing extra spaces
    clean_string = re.sub(' +', ' ', temp_string).lower()
    for each_word in clean_string.split():
        if each_word not in stop_words and len(each_word)>= 2:
        #if each_word not in final_stop_words and len(each_word)>= 2:
            temp_sent.append(lemmatizer.lemmatize(each_word))
        else:
            temp_sent.append("")
    return temp_sent


clean_lines = []

for line in all_scripts['Clean_Line_all']:
    line = remove_punc(line)
    line = [i for i in line if i]
    
    # remove character names 
    line = [word for word in line if word not in character_names]
    
    clean_lines.append(line)
 
all_scripts['Clean_Line_list'] = clean_lines


In [21]:
all_lines = []
for liste in all_scripts['Clean_Line_list']:
    
    # merge list back into sentences
    liste = ' '.join(liste)
    all_lines.append(liste)

all_scripts['Clean_Line'] = all_lines
all_scripts.head()

,Era,Movie,Character,Line,Gender,Group,Words_n,Clean_Line_all,Clean_Line_list,Clean_Line
0,The Golden Age,Snow White and the Seven Dwarfs,narrator,"Once upon a time, there lived a lovely little ...",Male,Single,48,"once upon a time, there lived a lovely little ...","[upon, time, lived, lovely, little, named, vai...",upon time lived lovely little named vain wicke...
1,The Golden Age,Snow White and the Seven Dwarfs,narrator,"Each day, the vain Queen consulted her Magic M...",Male,Single,44,"each day, the vain queen consulted her magic m...","[day, vain, consulted, mirror, mirror, wall, f...",day vain consulted mirror mirror wall fairest ...
2,The Golden Age,Snow White and the Seven Dwarfs,grimhilde,"Slave in the Magic Mirror, come from the farth...",Female,Single,10,"slave in the magic mirror, come from the farth...","[slave, mirror, come, farthest, space]",slave mirror come farthest space
3,The Golden Age,Snow White and the Seven Dwarfs,grimhilde,"Through wind and darkness, I summon thee. Speak!",Female,Single,8,"through wind and darkness, i summon thee. speak!","[darkness, summon, thee, speak]",darkness summon thee speak
4,The Golden Age,Snow White and the Seven Dwarfs,grimhilde,Let me see thy face.,Female,Single,5,let me see thy face.,"[let, see, thy, face]",let see thy face


In [22]:
# drop songs 
all_scripts = all_scripts[all_scripts["Line"].str.contains("♫")==False]
all_scripts = all_scripts[all_scripts["Line"].str.contains("♪")==False]


In [23]:
all_scripts = all_scripts.reset_index(drop=True)

In [24]:
# create a dataset with only the female and male speech 
gender = ["Male", "Female"]
all_scripts_binary = all_scripts.loc[all_scripts['Gender'].isin(gender)].reset_index(drop=True)

In [25]:
# save data 
all_scripts.to_pickle("all_scripts_cleaned_total_nonbinary.pickle")
all_scripts_binary.to_pickle("all_scripts_cleaned_total.pickle")

In [13]:
# both (clean_line_all + clean_line): lower lines, no songs, no html, contractions are fixed, 
# repeated characters occuring three times or more with one character
# only male and female speech

# clean_line : lemmatized, removed stopwords + character names, only a-z english letters.
# a word should be at least 2 letters 